In [11]:
import numpy as np
import matplotlib.pyplot as plt
import polars as pl
import folium

### Dataset
+ To download the dataset, go the link below and download the 'Trajectory Data' and rename it 'T-Drive_Taxi_Trajectory'
+ https://www.microsoft.com/en-us/research/publication/t-drive-trajectory-data-sample/

In [1]:
taxis_numbers = 5
time_interval = "6h"
interval = 5

DATASET_FOLDER = "T-Drive_Taxi_Trajectory"

df = pl.DataFrame()

for i in range(1, taxis_numbers + 1):
    df_taxi = pl.scan_csv(
    f"{DATASET_FOLDER}/{i}.txt",
    has_header = False,
    with_column_names = lambda _: ['id','datetime', "longitude", "latitude"]).collect()
    df = pl.concat([df, df_taxi])

df.replace_column(1, df["datetime"].str.to_datetime("%Y-%m-%d %H:%M:%S"))
df = df.with_columns(
        pl.struct(["longitude", "latitude"]).alias("coordinates"))\
            .sort("datetime")\
            .group_by_dynamic("datetime", every=time_interval).agg([pl.col("id"), pl.col("coordinates")]
    )
df

NameError: name 'pl' is not defined

In [ ]:
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen',
 'gray', 'black', 'lightgray']

map = folium.Map(
    location=[39.9042, 116.4074],  # Beijing coordinates
    zoom_start=12,
    tiles="https://webst01.is.autonavi.com/appmaptile?style=7&x={x}&y={y}&z={z}",
    attr="高德地图 (Gaode)"
)

locations = df[:interval].to_numpy()

for timed_location in locations:
    datetime = timed_location[0]
    idx_taxi = timed_location[1]
    group_locations = timed_location[2]
    for idx, location in enumerate(group_locations):
        long = location[0]
        lat = location[1]
        folium.Marker([lat, long],
                      icon=folium.Icon(color=colors[idx_taxi[idx - 1]]),
                      popup=f"Time Group: {datetime}").add_to(map)
map.save(f"map_save/{taxis_numbers}_taxis_{time_interval}_{interval}_interval.html")
map